# 🎓 Introduction to Cross-Validation: The Problem It Solves

In [5]:
## 📚 1. Setup and Data Loading (with Feature Engineering)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# --- Data Loading ---
file_path = '../../datasets/Supplement_Sales_Weekly_Expanded.csv'
try:
    data = pd.read_csv(file_path) 
    print("Raw data loaded successfully. Shape:", data.shape)
except FileNotFoundError:
    print("Error: Please check the path to your prepared dataset.")
    # Exit or raise error if loading fails to prevent subsequent KeyErrors

# --- Feature Engineering (Based on your Steps 1 & 2) ---

# 1. Data Cleaning and Preparation (Temporal & Grouping)
data['Date'] = pd.to_datetime(data['Date'])
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data = data.drop(columns=['Category', 'Revenue', 'Location'], errors='ignore')

# Calculate the average monthly price (as per your notes)
product_data_grouped = data.groupby(['Product_Name', 'Year', 'Month']).agg(
    Price_Avg=('Price', 'mean'), # Calculate the average price
    Product_ID=('Product_Name', 'first')
).reset_index()

# Sort data chronologically for lag/time features
product_data_grouped = product_data_grouped.sort_values(by=['Product_Name', 'Year', 'Month']).reset_index(drop=True)

# Select a single product for simplicity in this demo
PRODUCT_ID = product_data_grouped['Product_Name'].unique()[0]
product_data = product_data_grouped[product_data_grouped['Product_Name'] == PRODUCT_ID].copy()


# 2. Creating Additional Features (Time & Lag)
product_data['Time_Index'] = np.arange(len(product_data)) + 1
product_data['Time_Index_Squared'] = product_data['Time_Index'] ** 2

# Seasonal Coding
product_data['Month_sin'] = np.sin(2 * np.pi * product_data['Month'] / 12)
product_data['Month_cos'] = np.cos(2 * np.pi * product_data['Month'] / 12)

# Lag Variables (using .shift())
product_data['Price_Lag_1'] = product_data['Price_Avg'].shift(1)
product_data['Price_Lag_3'] = product_data['Price_Avg'].shift(3)
product_data['Price_Lag_12'] = product_data['Price_Avg'].shift(12)

# Moving Averages (using .rolling())
product_data['Price_MA_6'] = product_data['Price_Avg'].rolling(window=6).mean().shift(1)
product_data['Price_MA_12'] = product_data['Price_Avg'].rolling(window=12).mean().shift(1)

# Drop initial rows with NaN values created by lags/MA
product_data = product_data.dropna().reset_index(drop=True)
print("Features engineered. Ready for modeling. Final shape:", product_data.shape)

# Define features (X) and target (y)
FEATURES = ['Year', 'Month', 'Month_sin', 'Month_cos', 'Time_Index', 'Time_Index_Squared', 
            'Price_Lag_1', 'Price_Lag_3', 'Price_Lag_12', 'Price_MA_6', 'Price_MA_12']
TARGET = 'Price_Avg'

X = product_data[FEATURES]
y = product_data[TARGET]

Raw data loaded successfully. Shape: (4384, 10)
Features engineered. Ready for modeling. Final shape: (51, 14)


# 🎓 2. The Problem Cross-Validation Solves (Overfitting)

We are predicting the average price of a vitamin supplement using a Random Forest model.

### 🧠 The Exam Analogy and Overfitting

Imagine your **Model is a student** and your **Data is the exam material**.

> If you test your model using the **same data** it used to learn (study), the model will get a great score. This is called **overfitting**—the model has simply **memorized the answers** instead of learning the general concepts.
>
> That great score is a **fake measure** of how well the model will perform on **new, unseen data** in the real world.

### ⚠️ The Problem with a Single Split

In our original project, we used a single **80/20 train-test split**, respecting the chronological order (training on older data, testing on newer data).

While respecting time is good, relying on **just one single split** has a major flaw:

1.  **Luck/Unluck:** If the 20% test period happens to contain unusual, scattered prices (noise), our final score will look worse than it really is.
2.  **No Confidence Interval:** We get one score (e.g., MAE = 0.50). We don't know if that score is stable or just a lucky/unlucky result from that specific time window.

**Cross-Validation fixes this by performing many fair, independent tests.**

In [6]:
## 📉 3. The Unreliable Single Train-Test Split

# 3.1. Single split (Chronological)
# Since this is time-series data, we split chronologically (80% for train, 20% for test)
split_point = int(len(X) * 0.80)
X_train, X_test = X[:split_point], X[split_point:]
y_train, y_test = y[:split_point], y[split_point:]

print(f"Train size: {len(X_train)} samples. Test size: {len(X_test)} samples.")


# 3.2. Train and Evaluate
model_single = RandomForestRegressor(n_estimators=100, random_state=42)
model_single.fit(X_train, y_train)

y_pred_single = model_single.predict(X_test)

mae_single = mean_absolute_error(y_test, y_pred_single)
print(f"\nSingle Train-Test MAE (Mean Absolute Error): ${mae_single:.3f}")



Train size: 40 samples. Test size: 11 samples.

Single Train-Test MAE (Mean Absolute Error): $4.998


# 3.3. Interpretation (Markdown Cell)
### Interpretation of Single Split

We achieved a Mean Absolute Error (MAE) of **[Insert MAE from above]**.

**Question:** Is this a robust score? What if we had tested on a different 20% period? We have no way to know.

This is why we need Cross-Validation.

## 🔁 4. Introducing K-Fold Cross-Validation

Instead of one single test, K-Fold CV gives our model **multiple, independent tests**.

We split the entire dataset into **K** equal pieces (folds). We then run **K** separate experiments, rotating which piece is used for testing:

| Experiment | Training Data | Testing Data (Validation) |
| :--- | :--- | :--- |
| **Fold 1** | Folds 2, 3, 4, 5 | **Fold 1** |
| **Fold 2** | Folds 1, 3, 4, 5 | **Fold 2** |
| ... | ... | ... |

**Note on Time Series:** For true time-series data, standard K-Fold is usually not correct because it mixes past and future data. However, for a basic introduction, we will use it here to demonstrate the *averaging* concept. (We will address the proper time-series CV in a later notebook).

In [9]:
## 📊 5. Implementing K-Fold CV (5 Folds)

from sklearn.model_selection import KFold, cross_val_score

# Use K=5 for this introduction
kf = KFold(n_splits=5, shuffle=False) # shuffle=False maintains the time order within folds, though the folds themselves are not strictly chronological splits.

# Re-initialize the model
model_cv = RandomForestRegressor(n_estimators=100, random_state=42)

# Use cross_val_score: Note that 'neg_mean_absolute_error' is used because scikit-learn
# treats scoring metrics as something to be maximized (higher is better).
# The negative sign converts the error into a "score."
cv_scores = cross_val_score(
    model_cv, 
    X, 
    y, 
    cv=kf, 
    scoring='neg_mean_absolute_error'
)

# Convert negative scores back to positive MAE errors
cv_maes = -cv_scores 

print("Individual MAE scores for each fold (test):")
print(cv_maes)

print(f"\nFinal CV Score (Average MAE): ${cv_maes.mean():.3f}")
print(f"Standard Deviation of MAE: {cv_maes.std():.3f}")

Individual MAE scores for each fold (test):
[7.61504818 4.311373   8.2899025  5.60471    4.758919  ]

Final CV Score (Average MAE): $6.116
Standard Deviation of MAE: 1.571


## **🌟 6\. Conclusion: A Reliable Score**

Now we can directly compare the result from the single, potentially unreliable test against the more robust Cross-Validation (CV) average.

| Metric | Single Split (80/20) Result | K-Fold CV (Average) Result |
| :---- | :---- | :---- |
| **MAE** | **$4.998** | **$6.116** |
| **Robustness** | Low (Based on one single test period) | High (Based on 5 different test periods) |

### **🧠 What These Results Tell Us**

1. **The Single Split was Optimistic (and Likely Unreliable):**  
   * Your initial single test score of **$4.998** was quite low. This suggests that the final 20% of the data used for the test might have been an **easier, less noisy period** for the model to predict.  
   * If you had relied only on that $4.998 score, you would have **overestimated** your model's real-world accuracy.  
2. **The CV Score is the Honest Grade:**  
   * The **Average MAE of $6.116** is the model's true, general performance. This score is much more trustworthy because it ensures that **every part of your data** has been used fairly for testing.  
3. **The Model's Performance Varies:**  
   * The **Individual MAE Scores** ranged from **$4.311** to **$8.290**.  
   * The **Standard Deviation of 1.571** shows that the model's prediction accuracy changes significantly depending on the time period it's tested on. A high standard deviation means the model is **not perfectly stable**.  
   * This is a strong sign that the model may be struggling with high **variance** (a form of instability), which is exactly what CV is designed to expose\!

**In summary, Cross-Validation gave your model an honest, overall grade of $6.116, revealing that its performance is less stable than the single initial test suggested.**

# 🎓 Introduction to Cross-Validation: The Problem It Solves

## 🔍 Concept

**Can We Trust a Single Test Score?** Cross-Validation solves the fundamental problem of unreliable model evaluation by testing on multiple data splits instead of just one.

---

## 💡 Key Points

### The Problem: Overfitting & Unreliable Evaluation

**The Student Analogy**:
- 📚 Model = Student studying for an exam
- 📝 Training Data = Study materials
- ✅ Test Data = The actual exam

**What Goes Wrong with Single Split**:
- Student memorizes answers instead of learning concepts (overfitting)
- Testing on same data = letting student grade their own homework
- One test period might be unusually easy or hard (luck/bad luck)
- No confidence interval - just one number

### Dataset & Setup
- **Data**: `Supplement_Sales_Weekly_Expanded.csv`
- **Samples**: 51 time-series observations (after feature engineering from 4,384 rows)
- **Model**: RandomForestRegressor (n_estimators=100)
- **Target**: Predict average monthly supplement price
- **Features**: Time indices, lags (1, 3, 12 months), moving averages (6, 12 months), seasonality (sin/cos)

### The Experiment: Single Split vs K-Fold CV

**Single Train-Test Split (80/20 chronological)**:
```
Training: 40 samples → Testing: 11 samples
Result: MAE = $4.998
Question: Is this reliable?
```

**K-Fold Cross-Validation (K=5)**:
```
5 independent tests, each using different data
Every sample gets tested exactly once
Result: Average MAE = $6.116 ± $1.571
```

---

## 📊 Results Comparison

### Single Split vs Cross-Validation

| Metric | Single Split (80/20) | K-Fold CV (5 folds) | Difference |
|--------|---------------------|---------------------|------------|
| **MAE** | **$4.998** | **$6.116** | +22% error |
| **Std Dev** | Unknown (only 1 test) | **±$1.571** | High variance |
| **Confidence** | ❌ Low (lucky period?) | ✅ High (5 tests) |
| **Robustness** | ❌ Based on 1 period | ✅ Based on 5 periods |

### Individual Fold Performance (K-Fold CV)

```
Fold 1: $7.62  ━━━━━━━━━━━━━━━ (worst)
Fold 2: $4.31  ━━━━━━━ (best)
Fold 3: $8.29  ━━━━━━━━━━━━━━━━━ (catastrophic)
Fold 4: $5.60  ━━━━━━━━━
Fold 5: $4.76  ━━━━━━━━

Average: $6.116
Range: $3.98 (from best to worst)
```

---

## 🎯 Key Findings & Insights

### 1. The Single Split Was Deceptively Good
✅ **Single Split**: MAE = $4.998 looked great!  
❌ **Reality**: That 20% test period was **unusually easy** to predict  
⚠️ **Risk**: Would have overestimated model quality by 22%

**What Happened**: The final 11 samples (20% test set) happened to have:
- Lower price volatility
- Smoother trends
- Fewer market anomalies

### 2. Cross-Validation Revealed the Truth
✅ **Average MAE**: $6.116 (realistic performance)  
✅ **Std Dev**: ±$1.571 (model is unstable!)  
✅ **Range**: $4.31 to $8.29 (huge variability)

**What This Means**:
- Model performs **inconsistently** across time periods
- Some periods are easy ($4.31), others catastrophic ($8.29)
- **High variance** in predictions = unreliable for production

### 3. The Model Has Stability Issues
**Standard Deviation of ±$1.571** is concerning:
- Represents 26% variability relative to average error
- Model's accuracy **depends heavily** on which time period it predicts
- Suggests the model hasn't learned robust, generalizable patterns

---

## 📈 Visualization: How CV Works

### Single Split (Unreliable)
```
Timeline: [═══════════════════════════════════════════════════════]
          |────────────80%────────────|──20%──|
          Training (40 samples)        Test (11 samples)
          
Result: MAE = $4.998 ← Based on ONE lucky test period
```

### K-Fold Cross-Validation (Reliable)
```
Fold 1: [══════Test══════][═════════Train═════════════════════]
Fold 2: [═══════════Train════][══Test══][═══════Train═════════]
Fold 3: [════════════════Train════════][══Test══][═══Train════]
Fold 4: [═════════════════════Train══════════][══Test══][Train]
Fold 5: [════════════════════════════Train════════════][══Test]

Result: Average MAE = $6.116 ← Based on 5 diverse test periods
        Every sample tested exactly once
```

---

## 🎓 The Learning Experience

### The "Exam Analogy" Explained

**❌ Bad Testing (Single Split)**:
- Like giving a student ONE practice test
- They might get lucky with easy questions
- You think they're brilliant, but they just got lucky

**✅ Good Testing (Cross-Validation)**:
- Like giving 5 different exams
- Student must perform well on ALL of them
- Average score is their TRUE ability level

### Why the Single Split Failed

**The 20% test set was like an easy exam**:
- Contained a stable price period (low volatility)
- Few market shocks or anomalies
- Model appeared better than it really was

**Cross-Validation tested ALL conditions**:
- Stable periods (Fold 2: $4.31)
- Volatile periods (Fold 3: $8.29)
- Average conditions (Fold 4/5: ~$5)

---

## ⚠️ Critical Insights

### What We Discovered

**1. Luck Matters in Single Splits**
> If we had only trusted the $4.998 single-split result, we would have deployed a model that's actually 22% worse in real-world conditions.

**2. Variability Reveals Instability**
> The ±$1.571 standard deviation shows the model is **brittle** - it works well in some market conditions but fails in others (like Fold 3's $8.29).

**3. Cross-Validation is Essential**
> For time-series data with limited samples (51 observations), testing on multiple periods is the ONLY way to know if your model generalizes.

---

## 🚀 Implications & Next Steps

### For This Model

**Current Status**:
- ✅ Cross-Validation implemented correctly
- ⚠️ Model shows high variance (unstable)
- ❌ Not production-ready without improvements

**Why High Variance Occurred**:
- **Limited Data**: Only 51 samples after feature engineering
- **Market Complexity**: Supplement prices driven by external factors
- **Model Architecture**: Random Forest may be overfit to training patterns

### Recommended Actions

**Short-term**:
1. ✅ **Use CV Score ($6.12)** not single split ($5.00) for reporting
2. ⏭️ Try simpler models (Linear Regression, Ridge) for comparison
3. ⏭️ Add more features (external market data, seasonality indicators)
4. ⏭️ Collect more data if possible (increase from 51 samples)

**Long-term**:
1. Implement **TimeSeriesSplit** (proper temporal CV - Notebook 03)
2. Try ensemble methods to reduce variance
3. Test neural networks if more data available
4. Add confidence intervals to predictions (±$1.57 error band)

---

## 📚 What This Notebook Teaches

### Core Concepts Demonstrated

1. **Overfitting Recognition**: Single split can hide overfitting
2. **Evaluation Reliability**: Multiple tests > one test
3. **Performance Variance**: Models perform differently across periods
4. **Honest Metrics**: Average of many tests = true performance

### Why This Matters

**Before Cross-Validation**:
- "Our model has $5 error - great!" ← False confidence
- Deploy to production
- Realize errors are actually $6-8 in real use
- Business loses trust in ML

**After Cross-Validation**:
- "Our model has $6.12 error with ±$1.57 variability" ← Honest
- Improve model before deployment
- Set realistic expectations ($5-8 error range)
- Business gets reliable predictions

---

## 🎯 Key Takeaways

### The Main Lesson
> **Never trust a single train-test split.** Always use Cross-Validation to get multiple, independent performance estimates. The average score is your model's TRUE capability.

### Specific Numbers to Remember
- **Single Split**: $4.998 (22% too optimistic)
- **CV Average**: $6.116 (honest performance)
- **CV Std Dev**: ±$1.571 (model is unstable)
- **Range**: $4.31 to $8.29 (huge variability)

### Why This Foundation Matters

This notebook sets up the entire project:
- **Notebook 01** (this one): Why CV matters
- **Notebook 02**: Stratified K-Fold for classification
- **Notebook 03**: TimeSeriesSplit for proper temporal validation ⭐
- **Notebook 04-05**: Hyperparameter tuning WITH CV

Without understanding this problem, you can't appreciate why Notebook 03's TimeSeriesSplit ($5.08 MAE) is the REAL winner.

---

## 📊 Final Comparison: The Truth Revealed

| Evaluation Method | MAE | Reliability | Decision |
|------------------|-----|-------------|----------|
| **Single Split** | $4.998 | ❌ Unreliable (lucky) | Would deploy bad model |
| **K-Fold CV** | $6.116 ± $1.571 | ⚠️ Reliable but shows instability | Know model needs work |
| **TimeSeriesSplit** (Notebook 03) | **$5.08 ± $0.92** | ✅ Most reliable & stable | **Best for deployment** |

**Conclusion**: Cross-Validation saved us from a 22% overestimation of model quality!

---

## 🎓 Educational Value

**This notebook is foundational because it:**
1. ✅ Shows the **danger** of single splits (false confidence)
2. ✅ Demonstrates **how CV works** (multiple independent tests)
3. ✅ Reveals **model instability** (high variance across folds)
4. ✅ Sets up the need for **proper time-series CV** (Notebook 03)
5. ✅ Teaches **honest evaluation** (report averages & std dev)

**Connection to Final Results**:
- This K-Fold CV ($6.12) was still not ideal (ignores temporal order)
- TimeSeriesSplit (Notebook 03) gave $5.08 - even better!
- Neural Network (Notebook 05) gave $8.38 - worse than all CV methods
- **Moral**: Proper CV reveals truth, single splits hide it

---

*This is why Cross-Validation is the foundation of reliable machine learning!* 🎯
